In [ ]:
import pandas as pd
import json

def create_dataframe(annotations_path):
    with open(annotations_path, 'r') as file:
        data = json.load(file)
    images = pd.DataFrame(data['images']).rename(columns={'id': 'image_id'})[['image_id', 'file_name']]
    categories = pd.DataFrame(data['categories'])[['id', 'name']]
    categories.rename(columns={'id': 'category_id'}, inplace=True)
    usecols = ['image_id', 'category_id']
    annotations = pd.DataFrame(data['annotations'])[usecols]
    dataframe = annotations.merge(categories, on='category_id').merge(images, on='image_id')[['file_name', 'name']]
    return dataframe

In [ ]:
train_df = create_dataframe('annotations.json')
train_df

,file_name,name
0,131094.jpg,soft-cheese
1,131094.jpg,ham-raw
2,131094.jpg,hard-cheese
3,131094.jpg,bread-wholemeal
4,131094.jpg,cottage-cheese
...,...,...
76486,117029.jpg,damson-plum
76487,117524.jpg,damson-plum
76488,117849.jpg,damson-plum
76489,123468.jpg,damson-plum


In [ ]:
import os
splits = ['train', 'validation']
for split in splits:
    root = f'working/dataset/{split}'
    for index, row in train_df.iterrows():
        directory_name = row['name']
        directory_path = os.path.join(root, directory_name)
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)

In [ ]:
validation_df = create_dataframe('annotations.json')
validation_df

,file_name,name
0,149022.jpg,espresso-with-caffeine
1,149022.jpg,dark-chocolate
2,167905.jpg,espresso-with-caffeine
3,121313.jpg,espresso-with-caffeine
4,153429.jpg,espresso-with-caffeine
...,...,...
1825,144675.jpg,oat-milk
1826,103273.jpg,soup-potato
1827,159922.jpg,red-cabbage
1828,011275.jpg,pasta-in-conch-form


In [ ]:
import os
import shutil
def copy_images_to_destination(base_dir, dataframe, split):
    images_dir = os.path.join(base_dir, 'images')

    for index, row in dataframe.iterrows():
        file_name = row['file_name']
        file_class = row['name']
        dest_dir = os.path.join('working', 'dataset', split, file_class)
        os.makedirs(dest_dir, exist_ok=True)
        source_path = os.path.join(images_dir, file_name)
        destination_path = os.path.join(dest_dir, file_name)
        shutil.copyfile(source_path, destination_path)
    print("Done copying images.")

In [ ]:
base_dir = 'public_training_set_release_2.0'
dataframe = train_df
copy_images_to_destination(base_dir, dataframe, 'train')

Done copying images.


In [ ]:
base_dir = 'public_validation_set_2.0'
dataframe = validation_df
copy_images_to_destination(base_dir, dataframe, 'validation')

Done copying images.


In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory as ImageDataset
train = ImageDataset(
    directory=r'dataset/train',
    label_mode='categorical',
    batch_size=32,
    image_size=(299, 299)
)

Found 70397 files belonging to 498 classes.


In [ ]:
validation = ImageDataset(
    directory=r'dataset/validation',
    label_mode='categorical',
    batch_size=32,
    image_size=(299, 299)
)

Found 1799 files belonging to 498 classes.


In [ ]:
element = validation.as_numpy_iterator().next()

In [ ]:
x, y = element
x[0].shape, y[0].shape

((299, 299, 3), (498,))

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    inception = InceptionResNetV2(include_top=False,
                          weights='imagenet')

    inception.trainable = False

    inputs = keras.Input(shape=(None, None, 3))

    x = preprocess_input(inputs)
    x = inception(x, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(498, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    model.summary()

    model.compile(optimizer=tf.keras.optimizers.Adam(l),
                      loss=tf.keras.losses.CategoricalCrossentropy(),
                      metrics=[tf.keras.metrics.CategoricalAccuracy()])

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 tf.math.truediv_5 (TFOpLam  (None, None, None, 3)     0         
 bda)                                                            
                                                                 
 tf.math.subtract_5 (TFOpLa  (None, None, None, 3)     0         
 mbda)                                                           
                                                                 
 inception_resnet_v2 (Funct  (None, None, None, 1536   54336736  
 ional)                      )                                   
                                                                 
 global_average_pooling2d_7  (None, 1536)              0         
  (GlobalAveragePooling2D)                                 

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)

In [ ]:
history = model.fit(train,
                    epochs=32,
                    validation_data=validation,
                    callbacks=[callback]
                   )

Epoch 1/32
2200/2200 [==============================] - 583s 252ms/step - loss: 4.0691 - categorical_accuracy: 0.2259 - val_loss: 3.9940 - val_categorical_accuracy: 0.2240
Epoch 2/32
2200/2200 [==============================] - 539s 245ms/step - loss: 3.5547 - categorical_accuracy: 0.2648 - val_loss: 3.9486 - val_categorical_accuracy: 0.2279
Epoch 3/32
2200/2200 [==============================] - 532s 242ms/step - loss: 3.3576 - categorical_accuracy: 0.2830 - val_loss: 3.9249 - val_categorical_accuracy: 0.2324
Epoch 4/32
2200/2200 [==============================] - 540s 245ms/step - loss: 3.2212 - categorical_accuracy: 0.2964 - val_loss: 3.9433 - val_categorical_accuracy: 0.2340
Epoch 5/32
2200/2200 [==============================] - 545s 248ms/step - loss: 3.1143 - categorical_accuracy: 0.3088 - val_loss: 3.9777 - val_categorical_accuracy: 0.2424
Epoch 6/32
2200/2200 [==============================] - 554s 251ms/step - loss: 3.0253 - categorical_accuracy: 0.3178 - val_loss: 4.0223 - v